в качестве подопытного решил взть следующий датафрейм: https://www.kaggle.com/radmirzosimov/telecom-users-dataset  
по прогнозированию попадет ли клиент в отток или нет

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer

In [2]:
df = pd.read_csv("./data/telecom_users.csv")
df.drop(columns=["Unnamed: 0"], inplace=True)

In [3]:
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7010-BRBUU,Male,0,Yes,Yes,72,Yes,Yes,No,No internet service,...,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),24.10,1734.65,No
1,9688-YGXVR,Female,0,No,No,44,Yes,No,Fiber optic,No,...,Yes,No,Yes,No,Month-to-month,Yes,Credit card (automatic),88.15,3973.2,No
2,9286-DOJGF,Female,1,Yes,No,38,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.95,2869.85,Yes


In [4]:
#  посмотрим сколько наблюдений имеется
df.shape

(5986, 21)

In [5]:
#  поменяем целевую переменную на бинарный класс, вместо Yes, No чтобы потом не мучиться с этим
df.loc[df["Churn"] == "No", "Churn"] = 0
df.loc[df["Churn"] == "Yes", "Churn"] = 1

In [6]:
#  не такой уж большой фрейм, но в целом я думаю подойдет для тренировочных целей
#  посмотрим датафрейм на пропущенные значения
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [7]:
#  посмотрим на соотношение классов
df["Churn"].value_counts(normalize=True)

0    0.734881
1    0.265119
Name: Churn, dtype: float64

In [8]:
#  задача как обычно разбалансированная, но думаю для текущих целей не будем бороться с разбалансированностью классов

In [9]:
#  пропщенных значений нет, уже хорошо)) разделим на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, :"TotalCharges"], df["Churn"], test_size=0.3, 
                                                    random_state=42)

In [10]:
X_train.shape, y_train.shape

((4190, 20), (4190,))

In [11]:
X_test.shape, y_test.shape

((1796, 20), (1796,))

In [12]:
#  посмотрим поближе на датафрейм и сделаем пайплайн по его изменению
#  столбец customerID нам нафиг не нужен, поэтому в пайплайне его удалим 

#  столбцы gender, Partner, Dependents закодируем с помощью OrdinalEncoder, потому что данные фичи предоставляют бинарный класс
#  столбец SeniorCitizen оставим как есть, он и так уже реализует бинарный класс

#  далее идет два интересных столбца PhoneService и MultipleLines, которые взаимосвязаны. Поскольку данные категориальные 
#  и кодироваться они будут через OneHot, то попробуем их объединить. Потому что сейчас получается 5 вариантов OneHot, но можно
#  их уменьшить. (вообще по хорошему для всего того что я делаю нужно параллельно статтесты проводить чтобы доказывать
#  эти гипотезы, но предположим что я все это сделал и тесты дали добро на это все))) по столбцу PhoneService может быть два 
#  варианта Yes и No, а по столбцу MultipleLines можно наблюдать Yes, No и  No phone service. Логически данные в двух столбцах
#  пересекаются на уровнях No из первого столбца и No phone services из второго столбца, а Yes из первого столбца означает 
#  что клиент может иметь выделенные линии Yes или не иметь. Поэтому предлагаю смерджить два столбца и сделать один в котором 
#  будут следующие признаки. Yes - если клиент имеет телефонные сервисы, No - если клиент не имеет телефонные сервисы и multilines
#  если у клиента подключена опция выделенных линий...такой трансформер придется писать руками

#  фичу tenure предлагаю представить в виде логарифмически нормированного признака. Только надо не забыть, что там может быть 0

# посмотрим на следующие признаки [InternetService, OnlineSecurity, OnlineBackup, 
#                                  DeviceProtection, TechSupport,StreamingTV, StreamingMovies]

In [13]:
feature_review = pd.DataFrame()
for col in ["InternetService", "OnlineSecurity", "OnlineBackup", "DeviceProtection", 
            "TechSupport","StreamingTV", "StreamingMovies"]:
    temp_df = pd.DataFrame(df[col].value_counts())
    feature_review = feature_review.merge(temp_df, how="outer", left_index=True, right_index=True)
feature_review.fillna(0, inplace=True)

In [14]:
feature_review

,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
DSL,2068.0,0.0,0.0,0.0,0.0,0.0,0.0
Fiber optic,2627.0,0.0,0.0,0.0,0.0,0.0,0.0
No,1291.0,2982.0,2605.0,2640.0,2960.0,2389.0,2356.0
No internet service,0.0,1291.0,1291.0,1291.0,1291.0,1291.0,1291.0
Yes,0.0,1713.0,2090.0,2055.0,1735.0,2306.0,2339.0


In [15]:
#  как видно почти все колонки кроме InternerService имеют почити бинарные признаки. Как видно, отсутсвие подключения интернета
#  так же ведет за собой отсутствие прочих сервисов связанных с интернетом, что в целом логично. Поэтому предлагаю следюущее
#  столбец с интернет сервисом оставим без изменений, а остальные столбцы преобразуем к одному столбцу, в котором будет
#  одна бинарная фича.....есть дополнительные интернет услуги или нет. Потому что скорее всего, клиент который дополнительно
#  к услуге интернета подключает дополнительные услуги, навряд ли будет менять провайдера. Опять же это предположение
#  и по хорошему это надо проверять дополнительными тестами....в данном случае просто хочется потренироваться в написании 
#  пайплайнов))

In [16]:
#  по фиче Contract ничего делать не будем, потому что в принципе градация там нормальная и достаточно OneHot
#  колонки PaperlessBilling и PaymentMethod тоже взаимосвязаны, но образатывать их особо тоже не будем.
#  для PaperlessBilling OrdinalEncoder, а для PaymentMethod OneHot отлично подойдет
#  MonthlyCharges и TotalCharges сделаем дополнительно логарифмическое преобразование

In [17]:
#  напишем трансформеры

In [125]:
#  логарифмическое преобразование
#  плюс добавим стандартизирование исходной фичи
class PolyFeatureScaler(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        for col in self.col_names:
            out[f"{col}_log"] = np.where(out[col] == 0, 1, out[col])
            out[f"{col}_log"] = np.log(out[f"{col}_log"])
            
            out[col] = StandardScaler().fit_transform(out[[col]])
        return out

In [114]:
#  ручная подготовка датасета
class PrepareDF(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        out.drop(columns=["customerID"], inplace=True)
        out.loc[(out["PhoneService"] == "Yes") & (out["MultipleLines"] == "No"), "PhoneServ"] = "Phone"
        out.loc[(out["PhoneService"] == "Yes") & (out["MultipleLines"] == "Yes"), "PhoneServ"] = "MultilinePhone"
        out.loc[(out["PhoneService"] == "No") & (out["MultipleLines"] == "No phone service"), "PhoneServ"] = "NoPhone"
        out.drop(columns=["PhoneService", "MultipleLines"], inplace=True)
        
        out["ExtraInternetServ"] = pd.NA
        out.loc[(df["OnlineSecurity"] == "Yes") | 
                (df["OnlineBackup"] == "Yes") |
                (df["DeviceProtection"] == "Yes") |
                (df["TechSupport"] == "Yes") |
                (df["StreamingTV"] == "Yes") |
                (df["StreamingMovies"] == "Yes"), "ExtraInternetServ"] = 1
        out["ExtraInternetServ"].fillna(0, inplace=True)
        out.drop(columns=["OnlineSecurity", "OnlineBackup", "DeviceProtection", 
                          "TechSupport","StreamingTV", "StreamingMovies"], 
                 inplace=True)
        
#         в процессе выяснилось что в графе TotalCharges на самом деле текст и есть пропущенные значения в виде пробелов.
#         поэтому делаем замену, тоже ручками
        out.loc[out["TotalCharges"] == " ", "TotalCharges"] = 0
        out["TotalCharges"] = out["TotalCharges"].astype("float64")
    
        return out

In [95]:
#  Есть определенные проблемы с реализацией OrdinalEnoder в sklearn, чтобы ее обойти нужно дописать свой класс 
class OrdEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        df_for_replace = pd.DataFrame()
        for col in self.col_names:
            temp_col = OrdinalEncoder(categories="auto").fit_transform(df[[col]])
            temp_df = pd.DataFrame(temp_col, columns=[col])
            df_for_replace = pd.concat([df_for_replace, temp_df], axis=1)
        out.drop(columns=self.col_names, inplace=True)
        out = pd.concat([out, df_for_replace], axis=1)
        return out

In [101]:
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return pd.get_dummies(X, columns=self.col_names)

In [134]:
#  посмотрим как отрабатывает наш ручной преобразователь
PrepareDF().transform(X_train).head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,InternetService,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,PhoneServ,ExtraInternetServ
1187,Female,0,No,No,34,No,Month-to-month,No,Mailed check,20.35,673.20,Phone,0
5080,Female,0,Yes,No,72,No,Two year,No,Bank transfer (automatic),19.40,1496.45,Phone,0


In [133]:
#  посмотрим оставляет ли пропущенные значения
PrepareDF().transform(X_train).isna().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
InternetService      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges         0
PhoneServ            0
ExtraInternetServ    0
dtype: int64

In [30]:
#  нет, отлично, тогда поехали дальше, соберем трансформер как описано было раньше

In [129]:
transformer = make_pipeline(
                            PrepareDF(),
                            OrdEncoder(["gender", "Partner", "Dependents", "PaperlessBilling"]),
                            PolyFeatureScaler(["tenure", "MonthlyCharges", "TotalCharges"]),
                            OHEEncoder(["InternetService", "Contract", "PaymentMethod", "PhoneServ"])
                            )

In [135]:
transformer.fit_transform(X_train).head(2)

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,ExtraInternetServ,gender,Partner,Dependents,PaperlessBilling,tenure_log,...,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,PhoneServ_MultilinePhone,PhoneServ_NoPhone,PhoneServ_Phone
0,0.0,1.605839,-1.371498,-0.251750,0.0,1.0,1.0,1.0,0.0,4.276666,...,0,0,1,0,1,0,0,1,0,0
1,0.0,0.465966,0.765699,0.732798,1.0,0.0,0.0,0.0,1.0,3.784190,...,1,0,0,0,1,0,0,0,0,1


In [138]:
#  вроде отрабатывает правильно, после всех преобразований получилась 25 фичей при исходных 20....в общем неплохо, 
#  не наплодили лишнего, сделаем простой классификатор на деревьях решений

In [139]:
# ....continious will be soon